
# Inferencia, Tests de Hipótesis y A/B Testing para Data Science

**Objetivo del notebook:**  
Entender, con ejemplos simples y código, los conceptos básicos de:

- Hipótesis nula (H₀) y alternativa (H₁)  
- Estadístico de test  
- p-valor e interpretación  
- Test t de una muestra (media)  
- Comparación de dos grupos (A/B testing muy sencillo)  

Sin entrar en fórmulas avanzadas, todo con intuición y simulaciones.



## 0. Setup

Importamos librerías y fijamos semilla.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(123)

np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.precision', 3)

print("Librerías listas.")



## 1. Idea general de un test de hipótesis

En un test de hipótesis solemos:

1. Plantear una **hipótesis nula H₀** (ej: "la media es 0.5", "no hay diferencia entre grupos").  
2. Plantear una **hipótesis alternativa H₁** (ej: "la media es mayor que 0.5", "sí hay diferencia").  
3. Calcular un **estadístico de test** a partir de los datos.  
4. Ver qué tan extremo es ese estadístico si H₀ fuera cierta → **p-valor**.  
5. Decidir si **rechazamos** o **no rechazamos** H₀ (con un nivel de significación, por ejemplo α=0.05).

En Data Science, esta lógica se usa por ejemplo en:

- A/B tests de producto (¿la nueva versión mejora el CTR?).  
- Experimentos en marketing (¿un nuevo diseño aumenta ventas?).  
- Comparación de modelos (aunque ahí también se usan otras métricas).



## 2. Test t de una muestra (media)

Ejemplo:  
Tienes datos de tiempo que los usuarios pasan en tu app (en minutos al día).  
Quieres comprobar si el **tiempo medio** es realmente 10 minutos o si es diferente.

Formulamos:

- H₀: μ = 10  
- H₁: μ ≠ 10  (test bilateral)

Vamos a simular una muestra de usuarios.


In [ ]:

# Simulamos tiempos de uso (minutos/día)
true_mean = 11   # "realidad" (que no conocemos en la práctica)
true_sd = 3
n = 50

muestra = rng.normal(true_mean, true_sd, size=n)
muestra = np.clip(muestra, 0, None)

print("Media muestral:", muestra.mean())
print("Desviación muestral:", muestra.std(ddof=1))



### 2.1. Estadístico t (idea)

En un test t de una muestra, el estadístico (simplificando) es:

\[
t = \frac{\bar{x} - \mu_0}{s / \sqrt{n}}
\]

donde:

- \(\bar{x}\) es la media muestral  
- \(\mu_0\) es la media bajo H₀ (en nuestro caso, 10)  
- \(s\) es la desviación muestral  
- \(n\) es el tamaño de la muestra  

Cuanto más lejos esté \(\bar{x}\) de \(\mu_0\) en unidades de error estándar, más grande será |t| y más difícil será explicar ese resultado si H₀ fuera cierta.


In [ ]:

mu0 = 10  # media bajo H0

xbar = muestra.mean()
s = muestra.std(ddof=1)
n = len(muestra)

se = s / np.sqrt(n)
t_stat = (xbar - mu0) / se

print("Media muestral:", xbar)
print("Error estándar:", se)
print("Estadístico t:", t_stat)



### Ejercicio 1

1. Cambia el valor de `mu0` a 11 y recalcula el estadístico t.  
2. Observa cómo cambia su valor.  
3. ¿Qué intuición tienes? ¿Es más fácil o más difícil rechazar H₀ si μ₀ está más cerca de la media muestral?

*Resuélvelo cambiando el código anterior.*



## 3. p-valor por simulación (Monte Carlo) — Intuición

En vez de usar tablas de t de Student, podemos tener una **intuición de p-valor** simulando muchos datasets donde H₀ es cierta.

Idea:

- Suponemos que H₀ es verdad (μ = μ₀).  
- Generamos muchas muestras de tamaño n con media μ₀ y la misma desviación s.  
- Para cada muestra simulada, calculamos un t*.  
- Vemos qué proporción de |t*| es **tan grande o más** que el |t real| observado.  

Esa proporción ≈ p-valor (para test bilateral).


In [ ]:

# Aproximación del p-valor por simulación

num_sims = 5000
t_stats_sim = []

for _ in range(num_sims):
    sim_data = rng.normal(mu0, s, size=n)  # simulamos bajo H0 con sd≈s
    xbar_sim = sim_data.mean()
    s_sim = sim_data.std(ddof=1)
    se_sim = s_sim / np.sqrt(n)
    t_sim = (xbar_sim - mu0) / se_sim
    t_stats_sim.append(t_sim)

t_stats_sim = np.array(t_stats_sim)

# p-valor bilateral ≈ proporción de |t_sim| >= |t_obs|
p_val_aprox = np.mean(np.abs(t_stats_sim) >= np.abs(t_stat))
print("Estadístico t observado:", t_stat)
print("p-valor aproximado (bilateral):", p_val_aprox)

plt.figure()
plt.hist(t_stats_sim, bins=40)
plt.title("Distribución simulada de t bajo H0")
plt.xlabel("t* simulado"); plt.ylabel("frecuencia")
plt.show()



### Ejercicio 2

1. Interpreta el valor del p-valor aproximado.  
   - Si p < 0.05, diríamos que el resultado es **estadísticamente significativo** al 5%.  
2. Cambia el `true_mean` al principio (por ejemplo, a 10, 10.5, 12) y vuelve a ejecutar todo.  
3. Observa cómo se comportan la media muestral, t y el p-valor.



## 4. A/B Testing sencillo: comparación de dos medias

Supón que tienes dos versiones de una página:

- Versión A (actual)  
- Versión B (nueva)

Mides, por ejemplo, el **tiempo medio en la página** (o el importe de compra) en cada grupo.

Formulamos:

- H₀: μ_A = μ_B  (no hay diferencia)  
- H₁: μ_A ≠ μ_B  (sí hay diferencia)

Vamos a simular datos para A y B.


In [ ]:

# Simulamos datos A y B
n_A = 80
n_B = 75

true_mean_A = 50   # media real en A
true_mean_B = 52   # media real en B (ligeramente mayor)
sd_common = 10

A = rng.normal(true_mean_A, sd_common, size=n_A)
B = rng.normal(true_mean_B, sd_common, size=n_B)

print("Media A:", A.mean(), "| Desv A:", A.std(ddof=1))
print("Media B:", B.mean(), "| Desv B:", B.std(ddof=1))



### 4.1. Diferencia de medias y su error estándar

La diferencia de medias muestrales es:

\[
\bar{x}_B - \bar{x}_A
\]

Bajo H₀ (igualdad de medias), esta diferencia debería rondar 0 (salvo ruido).  
Su error estándar, si asumimos misma desviación, puede aproximarse por:

\[
SE_{\text{diff}} \approx \sqrt{\frac{s^2}{n_A} + \frac{s^2}{n_B}}
\]

donde \(s^2\) es una varianza "promedio" o se puede usar directamente las varianzas de cada grupo como aproximación.


In [ ]:

mean_A = A.mean()
mean_B = B.mean()
var_A = A.var(ddof=1)
var_B = B.var(ddof=1)

diff_mean = mean_B - mean_A

SE_diff = np.sqrt(var_A/n_A + var_B/n_B)

t_AB = diff_mean / SE_diff

print("Media A:", mean_A)
print("Media B:", mean_B)
print("Diferencia de medias B - A:", diff_mean)
print("SE de la diferencia:", SE_diff)
print("t AB:", t_AB)



### Ejercicio 3

1. ¿Tiene sentido el signo de la diferencia de medias (B - A) con cómo hemos simulado los datos?  
2. Cambia `true_mean_B` para que sea más cercano a `true_mean_A` y repite. ¿Qué le pasa al valor de t?



## 5. p-valor aproximado para el A/B test (por simulación)

De nuevo, podemos aproximar p-valor simulando bajo H₀.

Idea:

- Suponemos que **no hay diferencia real** → ambas muestras vienen de la misma distribución.  
- Para simular eso, mezclamos A y B y barajamos (permutación), o tomamos muestras de una misma normal con media combinada.  
- Calculamos diferencias de medias simuladas y vemos cuántas son tan extremas como la observada.


In [ ]:

# Método de permutación (shuffle) para aproximar p-valor

combined = np.concatenate([A, B])
n_A = len(A)
n_B = len(B)

num_sims = 3000
diff_sims = []

for _ in range(num_sims):
    rng.shuffle(combined)
    A_sim = combined[:n_A]
    B_sim = combined[n_A:]
    diff_sim = B_sim.mean() - A_sim.mean()
    diff_sims.append(diff_sim)

diff_sims = np.array(diff_sims)

p_val_perm = np.mean(np.abs(diff_sims) >= np.abs(diff_mean))

print("Diferencia observada B - A:", diff_mean)
print("p-valor aproximado (perm. bilateral):", p_val_perm)

plt.figure()
plt.hist(diff_sims, bins=40)
plt.title("Distribución de diferencias simuladas bajo H0 (A/B)")
plt.xlabel("diff_sim (B - A)"); plt.ylabel("frecuencia")
plt.show()



### Ejercicio 4

1. Si el p-valor es menor que 0.05, ¿qué concluirías sobre la versión B?  
2. Juega con distintos `true_mean_B` (más alto, igual, más bajo que A) y observa qué ocurre con el p-valor.  
3. Comenta si el tamaño de muestra (n_A, n_B) influye: prueba a ponerlos pequeños (por ejemplo 20).



## 6. Resumen y conexión con Data Science

En este notebook has visto:

- Cómo plantear H₀ y H₁.  
- Qué es un estadístico t y qué mide.  
- Qué significa un p-valor y cómo aproximarlo por simulación.  
- Cómo comparar una media con un valor objetivo (test de una muestra).  
- Cómo comparar dos grupos (A/B) con una diferencia de medias.

En muchas empresas, este tipo de razonamiento se aplica en:

- Experimentos de producto (¿lanzamos la versión B o no?).  
- Evaluar el impacto de un cambio en el funnel de conversión.  
- Ver si una campaña tiene efecto significativo en ventas o en uso.

Los modelos de Machine Learning suelen centrarse en la **predicción**, pero la estadística y los tests de hipótesis son clave para la **toma de decisiones**.
